In [4]:
%run "./Includes/Classroom-Setup"

### Managing the Complexity

Tools like Scikit-Learn and Spark ML use the abstraction of pipelines that bundle multiple stages together into a single object.  This works well as a first step in handling more complexity in the machine learning training process but has limitations in that these pipelines don't scale to support arbitrary code.  Rather, they normally only work with built-in functions in those packages.  Additionally, they do not allow for monitoring of each individual stage in that process.

MLflow allows for more robust pipelining by using multistep workflows.  This not only allows for a more flexible API but it allows teams to collaborate at higher level since different contributors can be responsible for different aspects of the pipeline.

**Multistep pipelines scales MLflow projects by allowing each stage to be its own project.**  The underlying idea that makes this possible is that **runs can recursively call other runs.**  This means that steps in a machine learning pipeline can be isolated.

There are a few options for passing artifacts between runs.  They can be saved to a common location or they can be logged as an artifact within MLflow.  This also allows for quicker iteration since runs can be run individually, you only need to rerun a given step if the last step's results have been cached.

Below is an example of a multistep workflow with four distinct steps.  

### Multistep Architecture

Now that we can package projects and run them in their environment, let's look at how  we can make workflows consisting of multiple steps.    There are three general architectures to consider:<br><br>

1. One driver project calls other entry points in that same project
2. One driver project calls other projects 
3. One project calls another project as its final step

In the last lesson, we used a Python file as a project that we ran.  In this lesson, we'll use Databricks notebooks instead.  These are all under the `Multistep` folder in your directory of notebooks.  There are two things to note about notebooks:<br><br>

0. Notebooks can be run using `dbutils.notebook.run("<path>", "<timeout>")`
0. Notebooks can be parameterized using widgets

Run the following code to see how this works.

In [ ]:
step1 = dbutils.notebook.run("./Multistep/Step-1-Read-Data", 60, 
  {"data_input_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv"})

print(step1)

This is the first step of our multistep workflow.  From the above, you can see that we ran the notebook `Step-1-Read-Data` in the `Multistep` directory.  We had a timeout of 60 seconds so if the code doesn't complete in that time, it would have failed.  Finally, we have the input path for our data.

Note the following is accomplished in four cells:<br><br>

0. Widgets are declared.  If you run these cells in the notebook itself, you'll see widgets appear at the top of the screen.  Widgets allow for the customization of notebooks without editing the code itself. They also allow for passing parameters into notebooks. 
0. Widgets are read.  This allows you to get the value of the widget and use it in your code.
0. An MLflow run logs the data from `data_input_path` as an artifact
0. The notebook exits and reports back information to the parent notebook (in our case, that's this notebook).

We can get data back from the executed notebook by parsing the exit string.

For debugging, you can either write your code in a notebook and then execute it as a run or you can click on `Notebook job #XXX` that appears after using `dbutils.notebook.run` to see where any problems might be.

In [14]:
import json

data_output_path = json.loads(step1).get("data_output_path")

print(data_output_path)

### Multistep Workflow

Now that we've created a single executable notebook with input parameters and output data, we can create more complex workflows. This notebook takes the data logged as an artifact and trains a model using specific hyperparameters.  The trained model is also logged.

In [16]:
step2 = dbutils.notebook.run("./Multistep/Step-2-Train", 60, 
  {"artifact_URI": data_output_path,
   "n_estimators": 10,
   "max_depth": 20,
   "max_features": "auto"})

print(step2)

Get the model output path from the result.

In [18]:
model_output_path = json.loads(step2).get("model_output_path")

print(model_output_path)

This step takes the saved model and creates predictions from it.  It logs those predictions as an artifact.

In [20]:
step3 = dbutils.notebook.run("./Multistep/Step-3-Predict", 60, 
  {"model_path": model_output_path,
   "data_path": data_output_path})

print(step3)

Parse the notebook output to see the predictions

In [22]:
predictions_output_path = json.loads(step3).get("predictions_output_path")

print(dbutils.fs.head(dbutils.fs.ls(predictions_output_path)[0].path))

## Review

**Question:** What are the benefits of pipelining?  
**Answer:** The biggest benefit to pipelining is managing the complexity of the machine learning process.  With loading, ETL, featurization, training, and prediction stages--to name a few--the complexity of the process quickly grows.  One additional benefit is collaboration since different individuals can work on different stages in the pipeline.

**Question:** How can I manage a pipeline using MLflow?  
**Answer:** Multi-step workflows chain together multiple MLflow jobs.  Runs can recursively call other runs.  This allows each stage to be its own MLflow project with its own environment, inputs, and outputs.

In [25]:
%run "./Includes/Classroom-Cleanup"